In [1]:
!pip install sumy

In [2]:
import pathlib
import textwrap
import requests

from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import pandas as pd

import re
import nltk
nltk.download('punkt')

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# API keys
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
SEARCH_API_KEY = 'YOUR_CUSTOM_SEARCH_ENGINE_API_KEY'
CX = 'YOUR_SEARCH_ENGINE_ID'
NUM = 50

genai.configure(api_key=GEMINI_API_KEY)

In [4]:
model = genai.GenerativeModel('gemini-pro')

In [5]:
# def transform_to_gemini(messages_chatgpt):
#     messages_gemini = []
#     system_promt = ''
#     for message in messages_chatgpt:
#         if message['role'] == 'system':
#             system_promt = message['content']
#         elif message['role'] == 'user':
#             messages_gemini.append({'role': 'user', 'parts': [message['content']]})
#         elif message['role'] == 'assistant':
#             messages_gemini.append({'role': 'model', 'parts': [message['content']]})
#     if system_promt:
#         messages_gemini[0]['parts'].insert(0, f"*{system_promt}*")

#     return messages_gemini

In [6]:
#  question = 'is modi a gopod leader'

# messages_chatgpt=[{"role": "system", "content": "Paraphrase the following question in different ways that preserve the core meaning but are different from each other in point of view, ideology etc. You can paraphrase in maximum four ways. Use bullet points for each category and not numbering."},
#           {"role": "user", "content": 'are chocolates good for health?'},
#           {"role": "assistant", "content": '''**Neutral**\n\n* Is the consumption of chocolate beneficial for overall health?\n* To what extent does chocolate contribute to one\'s well-being?\n* Are there any potential health benefits associated with the consumption of chocolate?\n\n**Positive**\n\n* Is chocolate a nutritious and health-promoting food?\n* How can the consumption of chocolate enhance one's physical and mental well-being?\n* What are the scientific studies that support the health benefits of chocolate?\n\n**Negative**\n\n* Is chocolate a harmful or dangerous substance for one's health?\n* In what ways does chocolate negatively impact the body and mind?\n* What are the potential risks associated with excessive chocolate consumption?\n\n**Balanced**\n\n* Are there both benefits and drawbacks to chocolate consumption?\n* To what extent do the potential benefits of chocolate outweigh the potential risks?\n* What are the nuances and complexities surrounding the health implications of chocolate?'''},
#           {"role": "user", "content": question} ]

In [7]:
# response = model.generate_content(transform_to_gemini(messages_chatgpt))

In [36]:
question = "are bananas good for children?"
response = model.generate_content(f"Paraphrase the following question in different ways that preserve the core meaning but are different from each other in point of view, ideology etc: '{question}'")

In [9]:
response.text

"**Scientific Perspective:**\n\n* Do bananas offer nutritional advantages that promote children's well-being?\n\n**Nutritional Perspective:**\n\n* How does the nutritional profile of bananas contribute to the dietary needs of children?\n\n**Health-Conscious Perspective:**\n\n* Are bananas a recommended fruit for children in terms of their impact on overall health?\n\n**Holistic Perspective:**\n\n* Can bananas support the physical, cognitive, and emotional development of children when included in their diets?\n\n**Parental Perspective:**\n\n* Are bananas a nutritious and convenient food option for busy parents to provide their children?\n\n**Dietary Ideology Perspective:**\n\n* From a particular dietary perspective (e.g., paleo, vegan), are bananas considered an appropriate fruit for children?\n\n**Environmental Perspective:**\n\n* Are bananas a sustainable and eco-friendly fruit choice for children based on their environmental impact?\n\n**Global Perspective:**\n\n* Do bananas provide 

In [26]:
def markdown_to_dict(markdown_text):

  # Remove the leading '> ' from each line.
  lines = markdown_text.split('\n')
  lines = [line[2:] for line in lines]
  # print(lines)

  # Split the lines into headings and queries.
  headings = []
  queries = []
  headings_pointer = -1
  result = {}
  for line in lines:
    if line.endswith('**'):
      headings_pointer += 1
      headings.append(line[:-2])
      result[headings[headings_pointer]] = []
    elif len(line) > 1:
      result[headings[headings_pointer]].append(line)

  return result

def google_search(query):
    # Base URL for Google Custom Search API
    base_url = 'https://www.googleapis.com/customsearch/v1'

    # Parameters for the API request
    params = {
        'key': SEARCH_API_KEY,
        'cx': CX,
        'q': query,
        'num':NUM,
        'lr': 'lang_en'
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return [{key: x[key] for key in ['title','link']} for x in data['items']]
    else:
        return [f"return code: {response.status_code} | response: {response}"]

def google_search_multiQ(questions):
    QnA = {}
    for q in questions:
        QnA[q] = google_search(q)
    return QnA

def get_website_content(url):
    try:
        parser = HtmlParser.from_url(url, Tokenizer('english'))
        return parser.document
    except Exception as e:
        return f"An error occurred: {str(e)}"

def get_content_summary(content):
    try:
        # Initialize the LsaSummarizer
        summarizer = LsaSummarizer()
        # Summarize the content using the LSA algorithm
        summary = summarizer(content, 3)  # Change the number for desired summary length
        # Join the sentences of the summary
        return ' '.join(map(str, summary))
    except Exception as e:
        return f"An error occurred: {str(e)}"

def add_summary_column(df):
    df['content'] = [get_website_content(url) for url in df['link']]
    df['summary'] = [get_content_summary(content) for content in df['content']]
    return df

dict_of_lists = markdown_to_dict(response.text)
print(dict_of_lists)


{'Scientific Perspective:': ["Do bananas offer nutritional advantages that promote children's well-being?"], 'Nutritional Perspective:': ['How does the nutritional profile of bananas contribute to the dietary needs of children?'], 'Health-Conscious Perspective:': ['Are bananas a recommended fruit for children in terms of their impact on overall health?'], 'Holistic Perspective:': ['Can bananas support the physical, cognitive, and emotional development of children when included in their diets?'], 'Parental Perspective:': ['Are bananas a nutritious and convenient food option for busy parents to provide their children?'], 'Dietary Ideology Perspective:': ['From a particular dietary perspective (e.g., paleo, vegan), are bananas considered an appropriate fruit for children?'], 'Environmental Perspective:': ['Are bananas a sustainable and eco-friendly fruit choice for children based on their environmental impact?'], 'Global Perspective:': ['Do bananas provide similar nutritional benefits for

In [11]:
print(dict_of_lists.keys())

dict_keys(['Scientific Perspective:', 'Nutritional Perspective:', 'Health-Conscious Perspective:', 'Holistic Perspective:', 'Parental Perspective:', 'Dietary Ideology Perspective:', 'Environmental Perspective:', 'Global Perspective:', 'Consumer Perspective:'])


In [12]:
rdf = pd.DataFrame([{"category": k, "question": s} for k,v in dict_of_lists.items() for s in v])

In [13]:
rdf

,category,question
0,Scientific Perspective:,Do bananas offer nutritional advantages that p...
1,Nutritional Perspective:,How does the nutritional profile of bananas co...
2,Health-Conscious Perspective:,Are bananas a recommended fruit for children i...
3,Holistic Perspective:,"Can bananas support the physical, cognitive, a..."
4,Parental Perspective:,Are bananas a nutritious and convenient food o...
5,Dietary Ideology Perspective:,"From a particular dietary perspective (e.g., p..."
6,Environmental Perspective:,Are bananas a sustainable and eco-friendly fru...
7,Global Perspective:,Do bananas provide similar nutritional benefit...
8,Consumer Perspective:,How do children perceive the taste and appeal ...


In [14]:
tdf = rdf

In [15]:
# tdf = pd.melt(pd.DataFrame(dict_of_lists), var_name='category', value_name='question')
# tdf

In [16]:
list(tdf['question'])

["Do bananas offer nutritional advantages that promote children's well-being?",
 'How does the nutritional profile of bananas contribute to the dietary needs of children?',
 'Are bananas a recommended fruit for children in terms of their impact on overall health?',
 'Can bananas support the physical, cognitive, and emotional development of children when included in their diets?',
 'Are bananas a nutritious and convenient food option for busy parents to provide their children?',
 'From a particular dietary perspective (e.g., paleo, vegan), are bananas considered an appropriate fruit for children?',
 'Are bananas a sustainable and eco-friendly fruit choice for children based on their environmental impact?',
 'Do bananas provide similar nutritional benefits for children in different regions of the world, considering cultural and dietary differences?',
 'How do children perceive the taste and appeal of bananas, and how does this influence their consumption?']

In [17]:
# Example
search = google_search_multiQ(list(tdf['question']))

In [18]:
list(search.keys())

["Do bananas offer nutritional advantages that promote children's well-being?",
 'How does the nutritional profile of bananas contribute to the dietary needs of children?',
 'Are bananas a recommended fruit for children in terms of their impact on overall health?',
 'Can bananas support the physical, cognitive, and emotional development of children when included in their diets?',
 'Are bananas a nutritious and convenient food option for busy parents to provide their children?',
 'From a particular dietary perspective (e.g., paleo, vegan), are bananas considered an appropriate fruit for children?',
 'Are bananas a sustainable and eco-friendly fruit choice for children based on their environmental impact?',
 'Do bananas provide similar nutritional benefits for children in different regions of the world, considering cultural and dietary differences?',
 'How do children perceive the taste and appeal of bananas, and how does this influence their consumption?']

In [20]:
search["Do bananas offer nutritional advantages that promote children's well-being?"]

['return code: 400 | response: <Response [400]>']

In [ ]:
expanded_data = []
for _, row in tdf.iterrows():
  for title, link in zip([item['title'] for item in search[row['question']]], [item['link'] for item in search[row['question']]]):
    # print(ans)
    expanded_data.append({'category': row['category'], 'question': row['question'], 'title': title, 'link': link})

expanded_df = pd.DataFrame(expanded_data)

expanded_df

In [27]:
expanded_df = pd.read_csv("expanded_df.csv")
expanded_df

,category,question,title,link,summary
0,Neutral:,Do chocolates possess any health benefits?,The Benefits of Having a Healthy Relationship ...,https://www.hopkinsmedicine.org/health/wellnes...,"Research shows that, unlike men, many women ha..."
1,Neutral:,Do chocolates possess any health benefits?,Chocolate Health Claims | NIH News in Health,https://newsinhealth.nih.gov/2021/02/chocolate...,She’s examining their impacts on short-term me...
2,Neutral:,Do chocolates possess any health benefits?,"Dark Chocolate: Health Benefits, Nutrients per...",https://www.webmd.com/diet/health-benefits-dar...,One review study found that eating dark chocol...
3,Positive:,Are there potential health advantages associat...,The Benefits of Having a Healthy Relationship ...,https://www.hopkinsmedicine.org/health/wellnes...,"Research shows that, unlike men, many women ha..."
4,Positive:,Are there potential health advantages associat...,"Chocolate: Health benefits, facts, and research",https://www.medicalnewstoday.com/articles/270272,The researchers set out to investigate whether...
5,Positive:,Are there potential health advantages associat...,7 Proven Health Benefits of Dark Chocolate,https://www.healthline.com/nutrition/7-health-...,"According to research, the polyphenols in dark..."
6,Negative:,What are the potential negative consequences o...,Health benefits and risks of chocolate,https://www.medicalnewstoday.com/articles/270272,The researchers set out to investigate whether...
7,Negative:,What are the potential negative consequences o...,Negative Side Effects of Chocolate | livestrong,https://www.livestrong.com/article/392195-choc...,An April 2017 paper in the Journal of Food Pro...
8,Negative:,What are the potential negative consequences o...,"COCOA: Overview, Uses, Side Effects, Precautio...",https://www.webmd.com/vitamins/ai/ingredientmo...,"Bozbulut R, Sanlier N, Döger E, Bideci A, Çamu..."


In [ ]:
# expanded_df.to_csv("expanded_df.csv",index=False)

In [22]:
# df_with_summary = add_summary_column(expanded_df)

In [29]:
df_with_summary = expanded_df
df_with_summary

,category,question,title,link,summary
0,Neutral:,Do chocolates possess any health benefits?,The Benefits of Having a Healthy Relationship ...,https://www.hopkinsmedicine.org/health/wellnes...,"Research shows that, unlike men, many women ha..."
1,Neutral:,Do chocolates possess any health benefits?,Chocolate Health Claims | NIH News in Health,https://newsinhealth.nih.gov/2021/02/chocolate...,She’s examining their impacts on short-term me...
2,Neutral:,Do chocolates possess any health benefits?,"Dark Chocolate: Health Benefits, Nutrients per...",https://www.webmd.com/diet/health-benefits-dar...,One review study found that eating dark chocol...
3,Positive:,Are there potential health advantages associat...,The Benefits of Having a Healthy Relationship ...,https://www.hopkinsmedicine.org/health/wellnes...,"Research shows that, unlike men, many women ha..."
4,Positive:,Are there potential health advantages associat...,"Chocolate: Health benefits, facts, and research",https://www.medicalnewstoday.com/articles/270272,The researchers set out to investigate whether...
5,Positive:,Are there potential health advantages associat...,7 Proven Health Benefits of Dark Chocolate,https://www.healthline.com/nutrition/7-health-...,"According to research, the polyphenols in dark..."
6,Negative:,What are the potential negative consequences o...,Health benefits and risks of chocolate,https://www.medicalnewstoday.com/articles/270272,The researchers set out to investigate whether...
7,Negative:,What are the potential negative consequences o...,Negative Side Effects of Chocolate | livestrong,https://www.livestrong.com/article/392195-choc...,An April 2017 paper in the Journal of Food Pro...
8,Negative:,What are the potential negative consequences o...,"COCOA: Overview, Uses, Side Effects, Precautio...",https://www.webmd.com/vitamins/ai/ingredientmo...,"Bozbulut R, Sanlier N, Döger E, Bideci A, Çamu..."


In [30]:
df_with_summary['summary'][3]

'Research shows that, unlike men, many women have feelings of guilt when eating this potentially “forbidden” food, or they fight the urge to eat it at all. This may be related to dark chocolate’s effects on heart health, since stress is a risk factor for cardiovascular disease. Researchers have not come up with a hard and fast recommendation of how much dark chocolate should be consumed to achieve these health benefits.'

In [31]:
df_with_summary['question'][3]

'Are there potential health advantages associated with eating chocolates?'

In [35]:
# concatenating the summaries for different perspectives.
summary_stuff = ''
for _, row in df_with_summary.iterrows():
  summary_stuff += f'{row["category"]}\t{row["summary"]}\n'
print(summary_stuff)

Neutral:	Research shows that, unlike men, many women have feelings of guilt when eating this potentially “forbidden” food, or they fight the urge to eat it at all. This may be related to dark chocolate’s effects on heart health, since stress is a risk factor for cardiovascular disease. Researchers have not come up with a hard and fast recommendation of how much dark chocolate should be consumed to achieve these health benefits.
Neutral:	She’s examining their impacts on short-term memory, focus, and overall brain function. But it’s too soon to tell whether cocoa supplements are beneficial for brain health. “So imagine, if this works for both heart health and cognition—or just one—this would be a very simple supplement that people could add to their diet.”
Neutral:	One review study found that eating dark chocolate helped to reduce blood pressure, though in small amounts. The American Heart Association recommends that you consume no more than 25-36 grams of added sugar per day, depending 

In [42]:
summary_stuff = """Neutral:	Research shows that, unlike men, many women have feelings of guilt when eating this potentially “forbidden” food, or they fight the urge to eat it at all. This may be related to dark chocolate’s effects on heart health, since stress is a risk factor for cardiovascular disease. Researchers have not come up with a hard and fast recommendation of how much dark chocolate should be consumed to achieve these health benefits.
Neutral:	She’s examining their impacts on short-term memory, focus, and overall brain function. But it’s too soon to tell whether cocoa supplements are beneficial for brain health. “So imagine, if this works for both heart health and cognition—or just one—this would be a very simple supplement that people could add to their diet.”
Neutral:	One review study found that eating dark chocolate helped to reduce blood pressure, though in small amounts. The American Heart Association recommends that you consume no more than 25-36 grams of added sugar per day, depending on your gender and size. These heavy metals can cause health problems, ranging from stomach irritation to developmental delays, in children and adults.
Negative:	The researchers set out to investigate whether chocolate bars containing plant sterols (PS) and cocoa flavanols (CF) have any effect on cholesterol levels. Scientists at Harvard Medical School have suggested that drinking two cups of hot chocolate a day could help keep the brain healthy and reduce memory decline in older people. Migraine risk: Some people may experience an increase in migraines when eating chocolate regularly due to cocoa’s tyramine, histamine, and phenylalanine content.
Negative:	An April 2017 paper in the Journal of Food Protection showed that many chocolate bars contain undeclared allergens like milk and nuts. Unfortunately, children around the world may get unacceptable exposure to heavy metals from eating chocolate. Correlations indicated that greater chocolate intake made it more likely a participant had cancer.
Negative:	Bozbulut R, Sanlier N, Döger E, Bideci A, Çamurdan O, Cinaz P. The effect of beta-glucan supplementation on glycemic control and variability in adolescents with type 1 diabetes mellitus. Decroix L, De Pauw K, Van Cutsem J, Pattyn N, Heyman E, Meeusen R. Acute cocoa flavanols intake improves cerebral hemodynamics while maintaining brain activity and cognitive performance in moderate hypoxia. Jenkins J, Williams D, Deng Y, et al. Eltrombopag, an oral thrombopoietin receptor agonist, has no impact on the pharmacokinetic profile of probe drugs for cytochrome P450 isoenzymes CYP3A4, CYP1A2, CYP2C9 and CYP2C19 in healthy men: a cocktail analysis.
Positive:	Research shows that, unlike men, many women have feelings of guilt when eating this potentially “forbidden” food, or they fight the urge to eat it at all. This may be related to dark chocolate’s effects on heart health, since stress is a risk factor for cardiovascular disease. Researchers have not come up with a hard and fast recommendation of how much dark chocolate should be consumed to achieve these health benefits.
Positive:	The researchers set out to investigate whether chocolate bars containing plant sterols (PS) and cocoa flavanols (CF) have any effect on cholesterol levels. Scientists at Harvard Medical School have suggested that drinking two cups of hot chocolate a day could help keep the brain healthy and reduce memory decline in older people. Migraine risk: Some people may experience an increase in migraines when eating chocolate regularly due to cocoa’s tyramine, histamine, and phenylalanine content.
Positive:	According to research, the polyphenols in dark chocolate may help lower some forms of LDL (“bad”) cholesterol when combined with other foods like almonds and cocoa ( 3). It’s possible that people who are already receiving treatment for high blood pressure may not get any additional benefit from adding cocoa flavanols to their diet. Additionally, cocoa contains stimulant substances like caffeine and theobromine, which may be a key reason why it can improve brain function in the short term ( 19)."""

In [43]:
question = "is chocolate good for health?"
final_response = model.generate_content(f"{summary_stuff}\n\nYou are an expert at answering questions with a balanced perspective. Above are some perspectives and their respective content to answer the question: {question}. Please closely analyse all the different viewpoints and crisply answer the question: {question} directly, but add the crisp information obtained from alternative perspectives to provide a more unbiased answer.")

In [44]:
final_response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': '**Is Chocolate Good for Health?**\n\nWhether chocolate is good or bad for health is a topic of ongoing research, with conflicting evidence.\n\n**Positive Perspectives:**\n\n* Dark chocolate contains polyphenols that may lower LDL cholesterol when consumed with other nutrient-rich foods.\n* Cocoa flavanols may improve brain function in the short term due to stimulants like caffeine and theobromine.\n* Hot chocolate may promote brain health and reduce memory decline in older adults.\n\n**Negative Perspectives:**\n\n* Some allergic reactions may be undeclared in chocolate bars.\n* Chocolate bars may contain heavy metals that can cause health issues.\n* Chocolate consumption may increase migraine frequency for some individuals.\n* The benefits of cocoa flavanols may not be significant for individuals already receiving high blood pressu

In [41]:
question = "is chocolate good for health?"
meh_final_response = model.generate_content(f"{question}")
print(meh_final_response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "**Health Benefits of Chocolate:**\n\n**1. Cardiovascular Health:**\n* Contains flavonoids, which improve blood flow, lower blood pressure, and reduce the risk of blood clots.\n* May help lower cholesterol levels.\n\n**2. Cognitive Function:**\n* Flavonoids improve blood flow to the brain, enhancing cognitive abilities, memory, and mood.\n* May protect against age-related cognitive decline.\n\n**3. Anti-Inflammatory Properties:**\n* Contains polyphenols, which have anti-inflammatory effects.\n* May help reduce inflammation throughout the body.\n\n**4. Antioxidant Properties:**\n* Rich in antioxidants that protect cells from damage caused by free radicals.\n* May help reduce the risk of chronic diseases such as heart disease and cancer.\n\n**5. Mood Enhancement:**\n* Contains theobromine and theophylline, which have stimulant effects